# MESH

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
import ujson
import sys
import os
import csv

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import defaultdict
from typing import Optional, Union

from bigbio.dataloader import BigBioConfigHelpers

sys.path.append('../../../..')
sys.path.append('..')
from DataModule import process_mention_dataset, process_umls_ontology, process_obo_ontology
from umls_utils import UmlsMappings
from bigbio_utils import CUIS_TO_REMAP, CUIS_TO_EXCLUDE, DATASET_NAMES, VALIDATION_DOCUMENT_IDS
from bigbio_utils import dataset_to_documents, dataset_to_df, resolve_abbreviation, get_left_context, get_right_context
from bioel.ontology import BiomedicalOntology

conhelps = BigBioConfigHelpers()

/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for an_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/an_em/an_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for anat_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/anat_em/anat_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True`

In [4]:
path_entity = '/home2/cye73/data_test2/arboel/bc5cdr/dictionary.pickle'
with open(path_entity, 'rb') as read_handle:
    dict = pickle.load(read_handle)
dict[0]

# path_entity2 = '/home2/cye73/arboEL2/data/arboel/bc5cdr/dictionary.pickle'
# with open(path_entity2, 'rb') as read_handle:
#     dict2 = pickle.load(read_handle)
# print("dict2 :\n", dict2[3])

{'cui': 'MESH:C000002',
 'title': 'bevonium',
 'type': 'CHEM',
 'description': 'bevonium ( CHEM : 2-(hydroxymethyl)-N,N-dimethylpiperidinium benzilate ; piribenzil methyl sulfate ; bevonium methylsulfate ; bevonium metilsulfate ; CG 201 ; Acabel ; bevonium sulfate (1:1) ; bevonium methyl sulfate )'}

In [9]:
# Define the path to your XML file
xml_file_path = '/home2/cye73/mesh/desc2024.xml'

counter = 0
# Open and read the file
with open(xml_file_path, 'r') as file:
    # Iterate through the first 500 lines
    for line_number, line in enumerate(file, start=1):
        
        if line_number > 1000000:  # Stop after 500 lines
            print(f"{line_number}: {line}", end='')  # Print line number and line
            counter += 1
            if counter > 1500 : 
                break



1000001:       </QualifierName>
1000002:     </QualifierReferredTo>
1000003:     <Abbreviation>UR</Abbreviation>
1000004:    </AllowableQualifier>
1000005:    <AllowableQualifier>
1000006:     <QualifierReferredTo>
1000007:      <QualifierUI>Q000821</QualifierUI>
1000008:       <QualifierName>
1000009:       <String>virology</String>
1000010:       </QualifierName>
1000011:     </QualifierReferredTo>
1000012:     <Abbreviation>VI</Abbreviation>
1000013:    </AllowableQualifier>
1000014:    <AllowableQualifier>
1000015:     <QualifierReferredTo>
1000016:      <QualifierUI>Q000662</QualifierUI>
1000017:       <QualifierName>
1000018:       <String>veterinary</String>
1000019:       </QualifierName>
1000020:     </QualifierReferredTo>
1000021:     <Abbreviation>VE</Abbreviation>
1000022:    </AllowableQualifier>
1000023:    <AllowableQualifier>
1000024:     <QualifierReferredTo>
1000025:      <QualifierUI>Q000266</QualifierUI>
1000026:       <QualifierName>
1000027:       <String>history<

In [3]:
import xml.etree.ElementTree as ET

# Define the path to your XML file
xml_file_path = '/home2/cye73/mesh/desc2024.xml'

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Helper function to safely get date components
def get_date_str(date_element):
    if date_element is not None:
        year = date_element.find("Year")
        month = date_element.find("Month")
        day = date_element.find("Day")
        # Build date string if all components are present
        return "/".join(part.text for part in [year, month, day] if part is not None)
    return None
    
def parse_descriptor_record(record):
    record_dict = {
        "DescriptorUI": record.find("DescriptorUI").text,
        "DescriptorName": record.find("DescriptorName/String").text,
    }

    # Applying the helper function for each date
    record_dict["DateCreated"] = get_date_str(record.find("DateCreated"))
    record_dict["DateRevised"] = get_date_str(record.find("DateRevised"))
    record_dict["DateEstablished"] = get_date_str(record.find("DateEstablished"))

    # Parsing AllowableQualifiersList as before
    allowable_qualifiers_list = []
    for qualifier in record.findall("AllowableQualifiersList/AllowableQualifier"):
        qualifier_dict = {
            "QualifierUI": qualifier.find("QualifierReferredTo/QualifierUI").text,
            "QualifierName": qualifier.find("QualifierReferredTo/QualifierName/String").text,
            "Abbreviation": qualifier.find("Abbreviation").text
        }
        allowable_qualifiers_list.append(qualifier_dict)
    record_dict["AllowableQualifiersList"] = allowable_qualifiers_list
    
    return record_dict


# List to hold dictionaries for each DescriptorRecord
descriptor_records = []

k = 0
# Iterate through each DescriptorRecord in the XML
for descriptor_record in root.findall("DescriptorRecord"):
    record_dict = parse_descriptor_record(descriptor_record)
    descriptor_records.append(record_dict)
    k += 1
    if k ==1 : 
        break

# Printing or processing the list of dictionaries
# For demonstration, print the first dictionary
print(descriptor_records[0])


{'DescriptorUI': 'D000001', 'DescriptorName': 'Calcimycin', 'DateCreated': '1974/11/19', 'DateRevised': '2023/02/26', 'DateEstablished': '1984/01/01', 'AllowableQualifiersList': [{'QualifierUI': 'Q000302', 'QualifierName': 'isolation & purification', 'Abbreviation': 'IP'}, {'QualifierUI': 'Q000276', 'QualifierName': 'immunology', 'Abbreviation': 'IM'}, {'QualifierUI': 'Q000493', 'QualifierName': 'pharmacokinetics', 'Abbreviation': 'PK'}, {'QualifierUI': 'Q000096', 'QualifierName': 'biosynthesis', 'Abbreviation': 'BI'}, {'QualifierUI': 'Q000627', 'QualifierName': 'therapeutic use', 'Abbreviation': 'TU'}, {'QualifierUI': 'Q000819', 'QualifierName': 'agonists', 'Abbreviation': 'AG'}, {'QualifierUI': 'Q000097', 'QualifierName': 'blood', 'Abbreviation': 'BL'}, {'QualifierUI': 'Q000134', 'QualifierName': 'cerebrospinal fluid', 'Abbreviation': 'CF'}, {'QualifierUI': 'Q000138', 'QualifierName': 'chemical synthesis', 'Abbreviation': 'CS'}, {'QualifierUI': 'Q000737', 'QualifierName': 'chemistry'